# **Assignment 1**

## Task 1

### Creating a project Structure

In [ ]:
# Implement libraries for creating folders and subfolders
import os 

# Main Directory
base_dir = "project"

# Subdirectories
subdirs = ["data", "results"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

print("Структура создана:", base_dir)

Структура создана: project


### Working with CSV files

In [1]:
# path to our csv file
path_to_csv = "/Users/asanali.tokenov/Desktop/Assignment_1_Assanali_Tokenov/project/data/students.csv"


# reading csv file
with open(path_to_csv, "r", encoding="utf-8") as f:
    data = f.read()
print("Results of reading the data", data)


Results of reading the data full_name,group,math,physics,cs,history,english
Shomanov Ayaulym,CS-23-01,56,71,69,85,84
Sadykova Moldir,CS-23-02,62,77,57,69,79
Sadykova Aliya,CS-23-03,68,83,80,69,82
Imanbekov Yerlan,AI-24-01,53,87,84,74,66
Nurmaganbet Temirlan,AI-24-02,63,75,75,74,71
Kydyr Dana,SE-22-01,78,64,92,61,72
Mukan Ayaulym,SE-22-02,90,75,68,61,84
Turganbek Temirlan,CS-23-01,96,89,89,75,78
Zhaksylykov Karina,CS-23-02,66,74,84,85,84
Mukan Rustem,CS-23-03,83,80,70,78,71
Ilyasov Sanzhar,AI-24-01,84,69,73,86,59
Mansurova Karina,AI-24-02,87,76,60,91,81
Nurmaganbet Madi,SE-22-01,69,79,90,83,65
Kapanova Aigerim,SE-22-02,86,80,86,77,81
Kapanova Bibigul,CS-23-01,98,65,79,86,89
Ilyasov Aigerim,CS-23-02,66,84,82,75,82
Tleubayeva Yerkebulan,CS-23-03,70,81,93,84,64
Sadykova Moldir,AI-24-01,86,86,74,72,66
Kydyr Aliya,AI-24-02,90,71,79,67,91
Ilyasov Altynai,SE-22-01,72,83,82,66,86
Abdullayev Damir,SE-22-02,92,92,82,79,63
Yessimov Bibigul,CS-23-01,96,87,70,70,81
Rakhimzhanov Rustem,CS-23-02,85,88

In [ ]:
import pandas as pd
# calculation of average score for each
df = pd.read_csv(path_to_csv)

subject_cols = [col for col in df.columns if col not in ("full_name", "group")]
df["avg_score"] = df[subject_cols].mean(axis=1) # axis = 1 is mean of columns, axis = 0 mean of rows

print(df[["full_name", "group", "avg_score"]])

                full_name     group  avg_score
0        Shomanov Ayaulym  CS-23-01       73.0
1         Sadykova Moldir  CS-23-02       68.8
2          Sadykova Aliya  CS-23-03       76.4
3        Imanbekov Yerlan  AI-24-01       72.8
4    Nurmaganbet Temirlan  AI-24-02       71.6
..                    ...       ...        ...
145      Kaldybek Ayaulym  SE-22-01       80.6
146         Beketov Damir  SE-22-02       83.2
147         Kaldybek Madi  CS-23-01       80.2
148      Zhussupov Gulnaz  CS-23-02       75.0
149       Kaldybek Gulnaz  CS-23-03       83.0

[150 rows x 3 columns]


### Save results

In [28]:
import json
from pathlib import Path

out_dir = Path("/Users/asanali.tokenov/Desktop/Assignment_1_Assanali_Tokenov/project/results")
out_file = out_dir / "report.json"

out_dir.mkdir(parents=True, exist_ok=True)

if out_file.exists():
    ans = input(f"File '{out_file}' already existed. Re-record? (y/n): ").strip().lower()
    if ans not in ("y", "yes"):
        print("Saving is declined")
        raise SystemExit
    
cols_to_save = ["full_name", "group", "avg_score"] # columns to save
records = df[cols_to_save].to_dict(orient="records") # forms a list of verbal types

# converting the types to native Python types (json doesn't like numpy)
clean_records = []
for r in records:
    clean = {}
    for k, v in r.items():
        # NaN -> None
        if pd.isna(v):   # checking is it empty, if yes -> None, otherwise continue the loop
            clean[k] = None
            continue
        # numpy scalars .item() -> python scalar
        try:
            if hasattr(v, "item"):
                v = v.item()  # converting to Python type
        except Exception:
            pass
        clean[k] = v
    clean_records.append(clean)

with open(out_file, "w", encoding="utf-8") as f:
    json.dump(clean_records, f, ensure_ascii= False, indent= 2)

print(f"Results are saved in: {out_file}")

Results are saved in: /Users/asanali.tokenov/Desktop/Assignment_1_Assanali_Tokenov/project/results/report.json


### Archive and Work with Pathlib

In [29]:
import zipfile # archive to zip-file
from datetime import datetime  # need to complete task 5

zip_path = out_dir.parent / "results.zip"  # archive near to our directory results

with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in out_dir.rglob("*"):
        if p.is_file():
            # in archive we will have the next structure: results/...
            zf.write(p, arcname=p.relative_to(out_dir.parent))

print(f"Archive created: {zip_path}")

# 5) Pathlib: checking report.json, output the size and time of modification
if out_file.exists():
    st = out_file.stat()
    size_bytes = st.st_size
    modified_dt = datetime.fromtimestamp(st.st_mtime)
    print(f"report.json exists at: {out_file}")
    print(f"Size: {size_bytes} bytes ({size_bytes/1024:.2f} KB)")
    print(f"Last modified: {modified_dt:%Y-%m-%d %H:%M:%S}")
else:
    print(f"report.json not found at: {out_file}")


Archive created: /Users/asanali.tokenov/Desktop/Assignment_1_Assanali_Tokenov/project/results.zip
report.json exists at: /Users/asanali.tokenov/Desktop/Assignment_1_Assanali_Tokenov/project/results/report.json
Size: 13868 bytes (13.54 KB)
Last modified: 2025-09-24 19:44:35


## Task 2

In [33]:
from pathlib import Path

# Directory with txt files
texts_dir = Path("texts")
texts_dir.mkdir(parents=True, exist_ok=True)

# Creating 5 txt files with some words (in my case i took lyrics from song Die With a Smile)
sample_files = [texts_dir / f"name_{i}.txt" for i in range(1, 6)]
if not all(p.exists() for p in sample_files):
    samples = [
        "I just woke up from a dream",
        "Where you and I had to say goodbye",
        "And I don't know what it all means",
        "But since I survived, I realized",
        "If the world was ending"
    ]
    for p, text in zip(sample_files, samples):
        p.write_text(text, encoding="utf-8")

# Search
query = input("Введите поисковый запрос: ").strip()
if not query:
    print("Пустой запрос — нечего искать.")
else:
    query_lower = query.lower()
    matches = []
    for p in texts_dir.glob("*.txt"):
        try:
            content = p.read_text(encoding="utf-8").lower()
        except UnicodeDecodeError:
            continue  # skip, if file not in UTF-8
        if query_lower in content:
            matches.append(p.name)

    if matches:
        print("Найдено в файлах:")
        for name in sorted(matches):
            print(name)
    else:
        print("Поисковый запрос отсутствует в данных файлах.")


Поисковый запрос отсутствует в данных файлах.


## Task 3

In [31]:
from pathlib import Path
import re

p = Path("file.txt")
text = p.read_text(encoding="utf-8", errors="ignore")

latin_letters = len(re.findall(r"[A-Za-z]", text))      # latin words
words = len(re.findall(r"[A-Za-z]+", text))             # all words latin and so on
lines = len(text.splitlines())                          # strings

print("Latin letters:", latin_letters)
print("Words:", words)
print("Strings:", lines)


Latin letters: 236
Words: 47
Strings: 6


## Task 4

In [15]:
from pathlib import Path
import re

words_path = Path("words.txt")
forbidden_path = Path("forbidden_words.txt")

text = words_path.read_text(encoding="utf-8", errors="ignore")
forbidden = forbidden_path.read_text(encoding="utf-8", errors="ignore").split()

# trick from gpt: longer words first, so as not to "break" long matches with short ones
for w in sorted(set(forbidden), key=len, reverse=True):
    pattern = re.compile(re.escape(w), re.IGNORECASE)
    text = pattern.sub(lambda m: "*" * len(m.group(0)), text)

print(text)


x****** is a high-level, interpreted *******ming **nguage that has gained immense popu**rity in recent years. It was first created in the **te 1980s by Guido van Rossum, and since then, it has become one of the most widely used *******ming **nguages in the world.
****** is known for its simplicity and ease of use, which makes it an ideal **nguage for beginners. It has a straightforward syntax and requires less c**e than other *******ming **nguages, which means that developers can focus on solving problems rather than writing long lines of c**e. ******_******_*******ming
